# Problema de Negócio (Versão Avançada - Refatorada)

A "Fazenda VerdeVida" planeja o mix de culturas entre Soja (duas variedades) e Milho Safrinha, maximizando lucro com restrições agronômicas, climáticas e logísticas.

**Restrições Agronômicas:**
- Limites de Potássio (K) e Fósforo (P) no solo.
- Área cultivável limitada por compactação do solo.

**Restrições Logísticas:**
- Horas de operação de máquinas (tratores, colheitadeiras).
- Capacidade finita de armazenamento nos silos.

Objetivo: Maximizar lucro total respeitando essas restrições.

Esta versão está organizada em funções para melhor reutilização e suporte a mudanças nos parâmetros.

# Modelagem e Coleta de Dados (Expandida)

Simulação de dados para talhões com 3 culturas: Soja Resistente, Soja Produtiva e Milho Safrinha.

Parâmetros estimados:
- Demanda de nutrientes (Potássio, Fósforo) por hectare.
- Horas de máquina necessárias por hectare.
- Produtividade para cálculo de armazenagem.

In [1]:
import pandas as pd
import numpy as np

# Importações essenciais da biblioteca PuLP
from pulp import LpProblem, LpVariable, lpSum, LpMaximize, value, LpStatus

In [2]:
# --- Configurações Iniciais ---

# Parâmetros para geração de dados mockados
n_dados = 1000  # Número de talhões/registros a simular
seed = 42       # Seed para garantir a reprodutibilidade dos dados

# Preços de mercado (R$ por tonelada)
preco_soja = 2500.0
preco_milho = 1200.0

# Arquivo para salvar os resultados e logs
output_file = open("resultado_otimizacao.txt", "w")

In [3]:
# --- Funções de Preparação de Dados e Recursos ---

def generate_mock_data(n, seed):
    """Gera dados simulados para diferentes talhões e culturas."""
    np.random.seed(seed)
    culturas = ["Soja_Resistente", "Soja_Produtiva", "Milho_Safrinha"]
    data = {
        "cultura": np.random.choice(culturas, n, p=[0.4, 0.4, 0.2]),
        "custo_ha": np.random.normal(loc=1800, scale=300, size=n),
        "uso_agua_m3_ha": np.random.normal(loc=450, scale=50, size=n),
        "demanda_k_kg_ha": np.random.normal(loc=100, scale=15, size=n),
        "demanda_p_kg_ha": np.random.normal(loc=80, scale=10, size=n),
        "horas_maquina_ha": np.random.normal(loc=12, scale=2, size=n),
        "produtividade_ton_ha": np.random.normal(loc=4.5, scale=0.8, size=n),
    }
    df = pd.DataFrame(data)
    # Ajustes para diferenciar as culturas
    df.loc[df['cultura'] == "Soja_Resistente", 'produtividade_ton_ha'] *= 0.85
    df.loc[df['cultura'] == "Soja_Resistente", 'custo_ha'] *= 1.05
    df.loc[df['cultura'] == "Soja_Produtiva", 'produtividade_ton_ha'] *= 1.15
    df.loc[df['cultura'] == "Milho_Safrinha", 'produtividade_ton_ha'] *= 1.2
    df.loc[df['cultura'] == "Milho_Safrinha", 'custo_ha'] *= 0.8
    return df

def calculate_parameters(df, preco_soja, preco_milho):
    """Calcula os parâmetros médios por cultura a partir dos dados."""
    params = df.groupby("cultura").mean()
    params["receita_ha"] = 0
    params.loc["Soja_Resistente", "receita_ha"] = params.loc["Soja_Resistente", "produtividade_ton_ha"] * preco_soja
    params.loc["Soja_Produtiva", "receita_ha"] = params.loc["Soja_Produtiva", "produtividade_ton_ha"] * preco_soja
    params.loc["Milho_Safrinha", "receita_ha"] = params.loc["Milho_Safrinha", "produtividade_ton_ha"] * preco_milho
    params["lucro_ha"] = params["receita_ha"] - params["custo_ha"]
    return params

def setup_resources():
    """Define os recursos totais disponíveis para a fazenda."""
    resources = {
        "AREA_TOTAL_DISPONIVEL_HA": 10000,
        "AREA_NAO_COMPACTADA_HA": 4000,
        "ORCAMENTO_TOTAL_DISPONIVEL": 8000000,
        "AGUA_TOTAL_DISPONIVEL_M3": 2000000,
        "POTASSIO_DISPONIVEL_KG": 450000,
        "FOSFORO_DISPONIVEL_KG": 350000,
        "HORAS_MAQUINA_DISPONIVEIS": 50000,
        "CAPACIDADE_SILO_TON": 20000,
    }
    return resources

In [4]:
import pandas as pd

# Função para configurar o modelo de otimização (COM RESTRIÇÃO DE DIVERSIFICAÇÃO)
def setup_model(params, resources):
    # Extrair parâmetros
    l_sr = params.loc["Soja_Resistente", "lucro_ha"]
    l_sp = params.loc["Soja_Produtiva", "lucro_ha"]
    l_ms = params.loc["Milho_Safrinha", "lucro_ha"]
    c_sr = params.loc["Soja_Resistente", "custo_ha"]
    c_sp = params.loc["Soja_Produtiva", "custo_ha"]
    c_ms = params.loc["Milho_Safrinha", "custo_ha"]
    a_sr = params.loc["Soja_Resistente", "uso_agua_m3_ha"]
    a_sp = params.loc["Soja_Produtiva", "uso_agua_m3_ha"]
    a_ms = params.loc["Milho_Safrinha", "uso_agua_m3_ha"]
    k_sr, p_sr = params.loc["Soja_Resistente", ["demanda_k_kg_ha", "demanda_p_kg_ha"]]
    k_sp, p_sp = params.loc["Soja_Produtiva", ["demanda_k_kg_ha", "demanda_p_kg_ha"]]
    k_ms, p_ms = params.loc["Milho_Safrinha", ["demanda_k_kg_ha", "demanda_p_kg_ha"]]
    h_sr = params.loc["Soja_Resistente", "horas_maquina_ha"]
    h_sp = params.loc["Soja_Produtiva", "horas_maquina_ha"]
    h_ms = params.loc["Milho_Safrinha", "horas_maquina_ha"]
    prod_sr = params.loc["Soja_Resistente", "produtividade_ton_ha"]
    prod_sp = params.loc["Soja_Produtiva", "produtividade_ton_ha"]
    prod_ms = params.loc["Milho_Safrinha", "produtividade_ton_ha"]

    # Criar modelo
    modelo = LpProblem(name="Otimizacao_Mix_Culturas", sense=LpMaximize)

    # Variáveis
    x_sr = LpVariable(name="Hectares_Soja_Resistente", lowBound=0, cat="Continuous")
    x_sp = LpVariable(name="Hectares_Soja_Produtiva", lowBound=0, cat="Continuous")
    x_ms = LpVariable(name="Hectares_Milho_Safrinha", lowBound=0, cat="Continuous")

    # >>>>> INÍCIO DA MODIFICAÇÃO <<<<<
    #
    # NOVA RESTRIÇÃO: Forçar Diversificação de Culturas
    # Para garantir que o modelo não escolha apenas a cultura mais lucrativa,
    # definimos uma área mínima a ser plantada para cada uma.
    # Usamos uma porcentagem da área não compactada, que é o gargalo principal.
    percentual_minimo_por_cultura = 0.05 # 5% para cada cultura
    area_minima_ha = resources["AREA_NAO_COMPACTADA_HA"] * percentual_minimo_por_cultura

    print(f"Adicionando restrição de plantio mínimo: {area_minima_ha:.2f} ha por cultura.")
    try:
        output_file.write(f"Adicionando restrição de plantio mínimo: {area_minima_ha:.2f} ha por cultura.\n")
    except Exception:
        # Se output_file não estiver definido (por exemplo, usuário executa células isoladamente),
        # apenas ignore a gravação no arquivo para não quebrar o fluxo.
        pass

    modelo += (x_sr >= area_minima_ha, "Minimo_Soja_Resistente")
    modelo += (x_sp >= area_minima_ha, "Minimo_Soja_Produtiva")
    modelo += (x_ms >= area_minima_ha, "Minimo_Milho_Safrinha")
    #
    # >>>>> FIM DA MODIFICAÇÃO <<<<<

    # Função Objetivo: Maximizar o Lucro Total
    modelo += lpSum([l_sr * x_sr, l_sp * x_sp, l_ms * x_ms]), "Lucro_Total"

    # Restrições (as mesmas de antes)
    modelo += (
        x_sr + x_sp + x_ms <= resources["AREA_TOTAL_DISPONIVEL_HA"],
        "Restricao_Area_Total",
    )
    modelo += (
        x_sr + x_sp + x_ms <= resources["AREA_NAO_COMPACTADA_HA"],
        "Restricao_Area_Nao_Compactada",
    )
    modelo += (
        c_sr * x_sr + c_sp * x_sp + c_ms * x_ms
        <= resources["ORCAMENTO_TOTAL_DISPONIVEL"],
        "Restricao_Orcamento",
    )
    modelo += (
        a_sr * x_sr + a_sp * x_sp + a_ms * x_ms
        <= resources["AGUA_TOTAL_DISPONIVEL_M3"],
        "Restricao_Agua",
    )
    modelo += (
        k_sr * x_sr + k_sp * x_sp + k_ms * x_ms <= resources["POTASSIO_DISPONIVEL_KG"],
        "Restricao_Potassio",
    )
    modelo += (
        p_sr * x_sr + p_sp * x_sp + p_ms * x_ms <= resources["FOSFORO_DISPONIVEL_KG"],
        "Restricao_Fosforo",
    )
    modelo += (
        h_sr * x_sr + h_sp * x_sp + h_ms * x_ms
        <= resources["HORAS_MAQUINA_DISPONIVEIS"],
        "Restricao_Horas_Maquina",
    )
    modelo += (
        prod_sr * x_sr + prod_sp * x_sp + prod_ms * x_ms
        <= resources["CAPACIDADE_SILO_TON"],
        "Restricao_Armazenagem",
    )

    return modelo, (x_sr, x_sp, x_ms)
output_file.write("Iniciando configuração de parâmetros...\n")

print(f"Gerando dados mockados com {n_dados} registros e seed {seed}...")
output_file.write(f"Gerando dados mockados com {n_dados} registros e seed {seed}...\n")

# Gerar dados
df = generate_mock_data(n_dados, seed)

print("Dados mockados gerados com sucesso.")
output_file.write("Dados mockados gerados com sucesso.\n")

# DEBUG: mostrar informações e primeiras linhas do DataFrame (útil para inspeção)
print(f"Dataframe gerado com shape: {df.shape}")
output_file.write(f"Dataframe shape: {df.shape}\n")

print("Exibindo as primeiras 10 linhas do dataframe:")
print(df.head(10))
output_file.write("Primeiras 10 linhas do dataframe:\n" + df.head(10).to_string() + '\n\n')

# Exportar o dataframe completo para CSV (mais eficiente) e registrar um resumo no outputs.txt
print("Exportando o dataframe completo para outputs.csv (formato CSV)...")
output_file.write("Exportando o dataframe completo para outputs.csv (formato CSV)...\n")
# Salvar CSV completo (index=False para facilitar reuso)",

output_file.write("Resumo descritivo do dataframe (head + describe):\n")
output_file.write(df.head(10).to_string() + '\n\n')
output_file.write(df.describe().to_string() + '\n')

print("Calculando parâmetros médios e lucros...")
output_file.write("Calculando parâmetros médios e lucros...\n")

# Calcular parâmetros
params = calculate_parameters(df, preco_soja, preco_milho)

print("Parâmetros calculados.")
output_file.write("Parâmetros calculados.\n")

print("Configurando recursos disponíveis...")
output_file.write("Configurando recursos disponíveis...\n")

# Configurar recursos
resources = setup_resources()

print("Recursos configurados.")
output_file.write("Recursos configurados.\n")

print("Parâmetros médios calculados:")
print(params)
output_file.write("Parâmetros médios calculados:\n" + params.to_string() + '\n')

Gerando dados mockados com 1000 registros e seed 42...
Dados mockados gerados com sucesso.
Dataframe gerado com shape: (1000, 7)
Exibindo as primeiras 10 linhas do dataframe:
           cultura     custo_ha  uso_agua_m3_ha  demanda_k_kg_ha  \
0  Soja_Resistente  1945.975815      379.684127       113.117756   
1   Milho_Safrinha  1119.517354      445.844721        90.253522   
2   Soja_Produtiva  1914.059355      374.763981        81.951987   
3   Soja_Produtiva  1983.175724      488.002798        84.369334   
4  Soja_Resistente  2066.333991      454.121988        92.691956   
5  Soja_Resistente  2230.445929      377.122425        94.721180   
6  Soja_Resistente  2152.685479      434.539546        88.450059   
7   Milho_Safrinha  1550.203219      412.392180        80.558250   
8   Soja_Produtiva  1778.950287      465.958726        93.158188   
9   Soja_Produtiva  1301.711720      517.022522       102.721399   

   demanda_p_kg_ha  horas_maquina_ha  produtividade_ton_ha  
0        78.496

/tmp/ipykernel_43094/3547607529.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '9483.42658290786' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  params.loc["Soja_Resistente", "receita_ha"] = params.loc["Soja_Resistente", "produtividade_ton_ha"] * preco_soja


765

In [5]:
# Função para configurar o modelo de otimização
def setup_model(params, resources):
    # Extrair parâmetros
    l_sr = params.loc["Soja_Resistente", "lucro_ha"]
    l_sp = params.loc["Soja_Produtiva", "lucro_ha"]
    l_ms = params.loc["Milho_Safrinha", "lucro_ha"]
    c_sr = params.loc["Soja_Resistente", "custo_ha"]
    c_sp = params.loc["Soja_Produtiva", "custo_ha"]
    c_ms = params.loc["Milho_Safrinha", "custo_ha"]
    a_sr = params.loc["Soja_Resistente", "uso_agua_m3_ha"]
    a_sp = params.loc["Soja_Produtiva", "uso_agua_m3_ha"]
    a_ms = params.loc["Milho_Safrinha", "uso_agua_m3_ha"]
    k_sr, p_sr = params.loc["Soja_Resistente", ["demanda_k_kg_ha", "demanda_p_kg_ha"]]
    k_sp, p_sp = params.loc["Soja_Produtiva", ["demanda_k_kg_ha", "demanda_p_kg_ha"]]
    k_ms, p_ms = params.loc["Milho_Safrinha", ["demanda_k_kg_ha", "demanda_p_kg_ha"]]
    h_sr = params.loc["Soja_Resistente", "horas_maquina_ha"]
    h_sp = params.loc["Soja_Produtiva", "horas_maquina_ha"]
    h_ms = params.loc["Milho_Safrinha", "horas_maquina_ha"]
    prod_sr = params.loc["Soja_Resistente", "produtividade_ton_ha"]
    prod_sp = params.loc["Soja_Produtiva", "produtividade_ton_ha"]
    prod_ms = params.loc["Milho_Safrinha", "produtividade_ton_ha"]

    # Criar modelo
    modelo = LpProblem(name="Otimizacao_Mix_Culturas", sense=LpMaximize)

    # Variáveis
    x_sr = LpVariable(name="Hectares_Soja_Resistente", lowBound=0, cat="Continuous")
    x_sp = LpVariable(name="Hectares_Soja_Produtiva", lowBound=0, cat="Continuous")
    x_ms = LpVariable(name="Hectares_Milho_Safrinha", lowBound=0, cat="Continuous")

    # Função Objetivo: Maximizar o Lucro Total
    # A função objetivo é a soma do lucro por hectare (l_cultura) multiplicado
    # pela área plantada de cada cultura (x_cultura). O objetivo é encontrar
    # a combinação de áreas que maximiza este valor total.
    modelo += lpSum([l_sr * x_sr, l_sp * x_sp, l_ms * x_ms]), "Lucro_Total"

    # Restrições: Definem os limites de recursos e requisitos

    # Restrição de Área Total Disponível: A soma das áreas plantadas não pode
    # exceder a área total máxima da fazenda.
    modelo += (
        x_sr + x_sp + x_ms <= resources["AREA_TOTAL_DISPONIVEL_HA"],
        "Restricao_Area_Total",
    )

    # Restrição de Área Não Compactada: A soma das áreas plantadas (as culturas
    # em questão) não pode exceder o limite de área não compactada.
    modelo += (
        x_sr + x_sp + x_ms <= resources["AREA_NAO_COMPACTADA_HA"],
        "Restricao_Area_Nao_Compactada",
    )

    # Restrição Orçamentária: O custo total de produção (custo_ha * área) para todas
    # as culturas não pode ultrapassar o orçamento total disponível.
    modelo += (
        c_sr * x_sr + c_sp * x_sp + c_ms * x_ms
        <= resources["ORCAMENTO_TOTAL_DISPONIVEL"],
        "Restricao_Orcamento",
    )

    # Restrição de Água: O consumo total de água (uso_agua_m3_ha * área) para todas
    # as culturas não pode exceder o volume total de água disponível para irrigação.
    modelo += (
        a_sr * x_sr + a_sp * x_sp + a_ms * x_ms
        <= resources["AGUA_TOTAL_DISPONIVEL_M3"],
        "Restricao_Agua",
    )

    # Restrição de Potássio (K): A demanda total de Potássio (demanda_k_kg_ha * área)
    # para todas as culturas deve ser menor ou igual ao Potássio disponível (estoque/suprimento).
    modelo += (
        k_sr * x_sr + k_sp * x_sp + k_ms * x_ms <= resources["POTASSIO_DISPONIVEL_KG"],
        "Restricao_Potassio",
    )

    # Restrição de Fósforo (P): A demanda total de Fósforo (demanda_p_kg_ha * área)
    # para todas as culturas deve ser menor ou igual ao Fósforo disponível (estoque/suprimento).
    modelo += (
        p_sr * x_sr + p_sp * x_sp + p_ms * x_ms <= resources["FOSFORO_DISPONIVEL_KG"],
        "Restricao_Fosforo",
    )

    # Restrição de Horas-Máquina: O total de horas de máquina necessárias (horas_maquina_ha * área)
    # para todas as operações não pode ultrapassar a disponibilidade total de horas-máquina.
    modelo += (
        h_sr * x_sr + h_sp * x_sp + h_ms * x_ms
        <= resources["HORAS_MAQUINA_DISPONIVEIS"],
        "Restricao_Horas_Maquina",
    )

    # Restrição de Armazenagem (Silo): A produção total esperada (produtividade_ton_ha * área)
    # não pode exceder a capacidade máxima de armazenagem do silo (em toneladas).
    modelo += (
        prod_sr * x_sr + prod_sp * x_sp + prod_ms * x_ms
        <= resources["CAPACIDADE_SILO_TON"],
        "Restricao_Armazenagem",
    )

    return modelo, (x_sr, x_sp, x_ms)

In [6]:
# Função para resolver e imprimir resultados
def solve_and_print(modelo, variables, file=None):
    x_sr, x_sp, x_ms = variables
    
    # Resolver
    modelo.solve()
    
    status = LpStatus[modelo.status]
    text = f"Status da Solução: {status}"
    print(text)
    if file: file.write(text + '\n')
    
    if status == 'Optimal':
        text = "Plano de Plantio Ótimo:"
        print(text)
        if file: file.write(text + '\n')
        text = f"  - Plantar {x_sr.varValue:.2f} ha de Soja Resistente."
        print(text)
        if file: file.write(text + '\n')
        text = f"  - Plantar {x_sp.varValue:.2f} ha de Soja Produtiva."
        print(text)
        if file: file.write(text + '\n')
        text = f"  - Plantar {x_ms.varValue:.2f} ha de Milho Safrinha."
        print(text)
        if file: file.write(text + '\n')
        text = f"\nLucro Máximo Esperado: R$ {value(modelo.objective):,.2f}"
        print(text)
        if file: file.write(text + '\n')
    else:
        text = "Não foi possível encontrar uma solução ótima. Verifique as restrições."
        print(text)
        if file: file.write(text + '\n')
    
    return status

In [7]:
# Função para análise de sensibilidade (definida antes de ser chamada)
def sensitivity_analysis(modelo, status, file=None):
    """Imprime e registra preços sombra (dual values) das restrições do modelo.
    Parâmetros:
      modelo: modelo PuLP já resolvido
      status: status retornado por solve_and_print
      file: objeto arquivo (opcional) para gravar a saída
    """
    if status == 'Optimal':
        text = "\nPreço Sombra (Shadow Price) dos Recursos:"
        print(text)
        if file: file.write(text + '\n')
        text = "Isso nos diz qual é o gargalo REAL da nossa operação!\n"
        print(text)
        if file: file.write(text + '\n')

        # Percorrer restrições do modelo e obter preco sombra
        for nome, restricao in modelo.constraints.items():
            try:
                preco_sombra = restricao.pi
            except Exception:
                preco_sombra = None
            if preco_sombra is None:
                text = f"  - Recurso '{nome}': Preço Sombra não disponível"
            else:
                text = f"  - Recurso '{nome}': Preço Sombra = R$ {preco_sombra:.2f}"
            print(text)
            if file: file.write(text + '\n')

            if preco_sombra is not None and preco_sombra > 0:
                text = "    -> GARGALO! Cada unidade a mais aumentaria o lucro em R$ {:.2f}.".format(preco_sombra)
                print(text)
                if file: file.write(text + '\n')
                text = "       Decisão: Vale a pena investir para obter mais deste recurso."
                print(text)
                if file: file.write(text + '\n')
            else:
                text = "    -> NÃO É GARGALO. Este recurso não está sendo 100% utilizado."
                print(text)
                if file: file.write(text + '\n')
    else:
        text = "Análise de sensibilidade não aplicável."
        print(text)
        if file: file.write(text + '\n')


In [8]:
print("Configurando o modelo de otimização...")
output_file.write("Configurando o modelo de otimização...\n")

# Configurar e resolver o modelo
modelo, variables = setup_model(params, resources)

print("Modelo configurado. Resolvendo o problema de otimização...")
output_file.write("Modelo configurado. Resolvendo o problema de otimização...\n")

status = solve_and_print(modelo, variables, output_file)

Configurando o modelo de otimização...
Modelo configurado. Resolvendo o problema de otimização...
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/vinicius/Downloads/estudo/po/PL/venv/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/70a70b660e9f40e79677a804593ff3ad-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/70a70b660e9f40e79677a804593ff3ad-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 41 RHS
At line 50 BOUNDS
At line 51 ENDATA
Problem MODEL has 8 rows, 3 columns and 24 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 7 (-1) rows, 3 (0) columns and 21 (-3) elements
0  Obj -0 Dual inf 23589.032 (3)
3  Obj 42913853
Optimal - objective value 42913853
After Postsolve, objective 42913853, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 42913852.77 - 3 iterations time 0

In [9]:
print("Executando análise de sensibilidade...")
output_file.write("Executando análise de sensibilidade...\n")

# Tentar executar a análise; se a função não estiver definida no kernel (NameError),
# definimos uma versão de fallback e executamos novamente.
try:
    sensitivity_analysis(modelo, status, output_file)
except NameError:
    msg = "Função 'sensitivity_analysis' não encontrada no kernel. Definindo fallback e executando..."
    print(msg)
    output_file.write(msg + '\n')

    # Definição de fallback (idempotente)
    def sensitivity_analysis(modelo, status, file=None):
        if status == 'Optimal':
            text = "\nPreço Sombra (Shadow Price) dos Recursos:"
            print(text)
            if file: file.write(text + '\n')
            text = "Isso nos diz qual é o gargalo REAL da nossa operação!\n"
            print(text)
            if file: file.write(text + '\n')
            
            for nome, restricao in modelo.constraints.items():
                preco_sombra = getattr(restricao, 'pi', None)
                if preco_sombra is None:
                    text = f"  - Recurso '{nome}': Preço Sombra não disponível"
                else:
                    text = f"  - Recurso '{nome}': Preço Sombra = R$ {preco_sombra:.2f}"
                print(text)
                if file: file.write(text + '\n')

                if preco_sombra is not None and preco_sombra > 0:
                    text = "    -> GARGALO! Cada unidade a mais aumentaria o lucro em R$ {:.2f}.".format(preco_sombra)
                    print(text)
                    if file: file.write(text + '\n')
                    text = "       Decisão: Vale a pena investir para obter mais deste recurso."
                    print(text)
                    if file: file.write(text + '\n')
                else:
                    text = "    -> NÃO É GARGALO. Este recurso não está sendo 100% utilizado."
                    print(text)
                    if file: file.write(text + '\n')
        else:
            text = "Análise de sensibilidade não aplicável."
            print(text)
            if file: file.write(text + '\n')

    # Executar a análise novamente
    sensitivity_analysis(modelo, status, output_file)

Executando análise de sensibilidade...

Preço Sombra (Shadow Price) dos Recursos:
Isso nos diz qual é o gargalo REAL da nossa operação!

  - Recurso 'Restricao_Area_Total': Preço Sombra = R$ -0.00
    -> NÃO É GARGALO. Este recurso não está sendo 100% utilizado.
  - Recurso 'Restricao_Area_Nao_Compactada': Preço Sombra = R$ -0.00
    -> NÃO É GARGALO. Este recurso não está sendo 100% utilizado.
  - Recurso 'Restricao_Orcamento': Preço Sombra = R$ -0.00
    -> NÃO É GARGALO. Este recurso não está sendo 100% utilizado.
  - Recurso 'Restricao_Agua': Preço Sombra = R$ -0.00
    -> NÃO É GARGALO. Este recurso não está sendo 100% utilizado.
  - Recurso 'Restricao_Potassio': Preço Sombra = R$ -0.00
    -> NÃO É GARGALO. Este recurso não está sendo 100% utilizado.
  - Recurso 'Restricao_Fosforo': Preço Sombra = R$ -0.00
    -> NÃO É GARGALO. Este recurso não está sendo 100% utilizado.
  - Recurso 'Restricao_Horas_Maquina': Preço Sombra = R$ -0.00
    -> NÃO É GARGALO. Este recurso não está sen

In [10]:
# Fechar o arquivo de saída
output_file.close()